# prj js object
js object
const prj = {Id: 23,
    number: "023",
    title: "Contextual Q&A Chat with AI, End-to-End App",
    info: "A AI app that answers based on provided context via pdfs and with chat history.",
    subInfo: "Langchain, History Retreivers,streamlit, HuggingFace, GroQ, Llama",
    imgPath: thb[5],
    category: "cat-d",
    dataSource: "https://contextualqnaai-k4kibvvy8vspmxwbsx9vfv.streamlit.app/)",
}

Task: explain code into paragraphs briefly and at last provide code with proper comments.
Style: Academic
Tone: Professional and 1st person
Audience: 30-year old
Format: Text


#Step1
<p>Project intro</p>
      <h4></h4>
      <Code
        code={`
          
          `}
      />
      <p>
        <br />
        <br />
      </p>

<div className="d-block text-center">
        <img
          src={img02}
          alt="result1"
          style={{ height: "300px", width: "300px" }}
        />
</div>

<p>

<br>
<br>

</p>

In [ ]:
#Step1: AI Model
from dotenv import load_dotenv
load_dotenv()
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
groq_api_key = os.getenv('GROQ_API_KEY')
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

#Step2: RAG Chain
#Data Loadin, Data chunking, Documnet Embedding, VectorStores
loader=PyPDFLoader('test.pdf')
documents=loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever() 

<p>
In the first step, we use the dotenv library to load environment variables from a .env file. This is a secure way to manage sensitive information like API keys. WE then set these environment variables for HuggingFace and Groq API keys. The HuggingFace embeddings model (all-MiniLM-L6-v2) is initialized to convert text into numerical representations (embeddings). The ChatGroq language model (Llama3-8b-8192) is also initialized using the Groq API key.
<br/>

In the second step, We load a PDF document using the PyPDFLoader class. This document is then split into smaller chunks using the RecursiveCharacterTextSplitter, which ensures that each chunk is of a manageable size (5000 characters) with a slight overlap (200 characters) to maintain context. These chunks are then embedded using the HuggingFace embeddings model and stored in a FAISS (Facebook AI Similarity Search) vector store. This vector store is configured to act as a retriever, allowing efficient retrieval of relevant document chunks based on a query.
<br/>


</p>

In [ ]:
# Step 3: Prompts and history aware
# Chat history aware prompt

# System message for reformulating questions based on chat history
system_with_history_message = (
    "Given a chat history and the latest user question"
    " which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

# Create a chat prompt template with the system message and placeholders
system_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_with_history_message),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a history-aware retriever by combining LLM, retriever, and the history-aware prompt
history_aware_retriever = create_history_aware_retriever(llm, retriever, system_with_history_prompt)

# Actual system prompt for question-answering tasks
system_actual_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

# Create a chat prompt template for the actual system prompt
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_actual_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


<p>
In this step, we create prompts that are aware of the chat history to ensure that questions are contextually accurate and relevant. First, we set up a system message that instructs the model to reformulate a user’s question into a standalone question that can be understood without the chat history. This is achieved using the ChatPromptTemplate with a system message and placeholders for chat history and user input.
<br/>
Next, we create a history-aware retriever by combining the language model (LLM) and the retriever with the history-aware prompt. This ensures that the retriever can handle context from the chat history effectively.
Finally, we set up the actual system prompt for question-answering tasks. This prompt instructs the assistant to use retrieved context to answer questions concisely. The ChatPromptTemplate is used again to structure this prompt with placeholders for chat history and user input.
<br/>


</p>

In [ ]:

# Step 4: Runnable chains
# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create the Retrieval-Augmented Generation (RAG) chain
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
# Session management
store = {}
# Function to get session history
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Create the conversational RAG chain
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)
# Invoke the chain with a sample input and configuration
response = conversational_rag_chain.invoke(
    {"input": "what is the topic about?"},
    config={
        "configurable": {"session_id": "chat1"}
    }  
)

# Retrieve the answer from the response
response['answer']



'The topic is about the memory of World War I and diverse European narratives about the war.'

<p>
In this step, we focus on creating runnable chains to handle the question-answering process effectively. We begin by setting up a question_answer_chain using the create_stuff_documents_chain function. This chain leverages the language model (LLM) and the question-answering prompt (qa_prompt) to generate answers based on the provided context. This ensures that the assistant can produce accurate and relevant responses. Next, we create a rag_chain using the create_retrieval_chain function. This chain integrates the history-aware retriever with the question-answer chain, allowing for contextually relevant answers by considering the entire conversation history. This step is crucial for maintaining the flow and coherence of the conversation, ensuring that the assistant can retrieve and utilize relevant information effectively.
<br/>
We then manage session histories using a store dictionary. The get_session_history function retrieves the chat history for a given session ID, initializing a new ChatMessageHistory object if the session ID does not exist in the store. This setup allows us to maintain context across different sessions, enhancing the assistant’s ability to provide consistent and context-aware responses.Finally, we create a conversational_rag_chain using the RunnableWithMessageHistory class. This chain combines the RAG chain with the session history retrieval function, specifying keys for input messages, chat history, and output messages. By invoking this chain with a sample input question and a configuration that includes a session ID, we ensure that the assistant can handle conversations with context awareness. This setup allows the assistant to provide accurate and concise answers based on the chat history and retrieved context, enhancing the overall user experience.
<br/>


</p>

In [ ]:
#Step5

<p>

<br>
<br>

</p>